In [608]:
import pandas as pd 
import sasoptpy as so #optimization modeling package 
import requests 
import os 
import time 
import re 

In [609]:
#Data
r = requests.get('https://fantasy.premierleague.com/api/bootstrap-static/')
fpl_data = r.json()


In [610]:
fpl_data.keys()

dict_keys(['chips', 'events', 'game_settings', 'game_config', 'phases', 'teams', 'total_players', 'element_stats', 'element_types', 'elements'])

In [611]:
element_data = pd.DataFrame(fpl_data['elements'])

In [612]:
element_data.head()

player_name_map = dict(zip(element_data['id'], element_data['web_name']))
#print(dict(list(player_name_map.items())[:10]))  # Show first 10 mappings


In [613]:
team_data = pd.DataFrame(fpl_data['teams'])
team_data.head()


,code,draw,form,id,loss,name,played,points,position,short_name,strength,team_division,unavailable,win,strength_overall_home,strength_overall_away,strength_attack_home,strength_attack_away,strength_defence_home,strength_defence_away,pulse_id
0,3,0,None,1,0,Arsenal,0,0,0,ARS,4,None,False,0,1320,1325,1350,1350,1290,1300,1
1,7,0,None,2,0,Aston Villa,0,0,0,AVL,3,None,False,0,1125,1250,1110,1200,1140,1300,2
2,90,0,None,3,0,Burnley,0,0,0,BUR,2,None,False,0,1050,1050,1050,1050,1050,1050,43
3,91,0,None,4,0,Bournemouth,0,0,0,BOU,3,None,False,0,1150,1180,1100,1160,1200,1200,127
4,94,0,None,5,0,Brentford,0,0,0,BRE,3,None,False,0,1120,1185,1080,1080,1160,1290,130


In [614]:
element_data.loc[0].keys()

Index(['can_transact', 'can_select', 'chance_of_playing_next_round',
       'chance_of_playing_this_round', 'code', 'cost_change_event',
       'cost_change_event_fall', 'cost_change_start', 'cost_change_start_fall',
       'dreamteam_count', 'element_type', 'ep_next', 'ep_this', 'event_points',
       'first_name', 'form', 'id', 'in_dreamteam', 'news', 'news_added',
       'now_cost', 'photo', 'points_per_game', 'removed', 'second_name',
       'selected_by_percent', 'special', 'squad_number', 'status', 'team',
       'team_code', 'total_points', 'transfers_in', 'transfers_in_event',
       'transfers_out', 'transfers_out_event', 'value_form', 'value_season',
       'web_name', 'region', 'team_join_date', 'birth_date',
       'has_temporary_code', 'opta_code', 'minutes', 'goals_scored', 'assists',
       'clean_sheets', 'goals_conceded', 'own_goals', 'penalties_saved',
       'penalties_missed', 'yellow_cards', 'red_cards', 'saves', 'bonus',
       'bps', 'influence', 'creativity', 't

In [615]:
element_data.loc[0]['team_code']

np.int64(3)

In [616]:
elements_team = pd.merge(element_data, team_data, left_on = 'team', right_on = 'id' )



In [617]:
review_data = pd.read_csv('../data/input.csv')
review_data.head()

,assists,birth_date,bonus,bps,can_select,can_transact,chance_of_playing_next_round,chance_of_playing_this_round,clean_sheets,clean_sheets_per_90,code,corners_and_indirect_freekicks_order,corners_and_indirect_freekicks_text,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,creativity,creativity_rank,creativity_rank_type,direct_freekicks_order,direct_freekicks_text,dreamteam_count,element_type,ep_next,ep_this,event_points,expected_assists,expected_assists_per_90,expected_goal_involvements,expected_goal_involvements_per_90,expected_goals,expected_goals_conceded,expected_goals_conceded_per_90,expected_goals_per_90,first_name,form,form_rank,form_rank_type,goals_conceded,goals_conceded_per_90,goals_scored,has_temporary_code,ict_index,ict_index_rank,ict_index_rank_type,id,in_dreamteam,influence,influence_rank,influence_rank_type,minutes,mng_clean_sheets,mng_draw,mng_goals_scored,mng_loss,mng_underdog_draw,mng_underdog_win,mng_win,news,news_added,now_cost,now_cost_rank,now_cost_rank_type,opta_code,own_goals,penalties_missed,penalties_order,penalties_saved,penalties_text,photo,points_per_game,points_per_game_rank,points_per_game_rank_type,red_cards,region,removed,saves,saves_per_90,second_name,selected_by_percent,selected_rank,selected_rank_type,special,squad_number,starts,starts_per_90,status,team,team_code,team_join_date,threat,threat_rank,threat_rank_type,total_points,transfers_in,transfers_in_event,transfers_out,transfers_out_event,value_form,value_season,web_name,yellow_cards
0,0,NaN,0,0,False,True,0.0,0.0,0,0.00,438098,NaN,NaN,0,0,-1,1,0.0,795,343,NaN,NaN,0,3,0.0,0.0,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Fábio,0.0,773,331,0,0.00,0,False,0.0,795,343,1,False,0.0,795,343,0,0,0,0,0,0,0,0,Has joined Portuguese side FC Porto on loan for the 2024/25 season,2024-08-29T11:06:25.241953Z,54,134,82,p438098,0,0,NaN,0,NaN,438098.jpg,0.0,794,343,0,NaN,False,0,0.0,Ferreira Vieira,0.0,714,295,False,NaN,0,0.00,u,1,3,NaN,0.0,791,341,0,439,0,2825,0,0.0,0.0,Fábio Vieira,0
1,2,1997-04-03,6,152,False,True,0.0,0.0,2,0.30,205651,NaN,NaN,0,0,-5,5,119.5,276,31,NaN,NaN,2,4,0.0,0.0,0,0.52,0.08,3.57,0.54,3.05,5.82,0.87,0.46,Gabriel,0.0,795,83,5,0.75,3,False,52.6,287,33,2,False,154.4,349,33,600,0,0,0,0,0,0,0,Knee injury - Expected back 01 Jun,2025-01-12T22:00:07.802845Z,65,43,20,p205651,0,0,NaN,0,NaN,205651.jpg,2.5,216,25,0,30.0,False,0,0.0,Fernando de Jesus,1.0,200,32,False,NaN,6,0.90,i,1,3,2022-07-04,255.0,134,32,42,1238437,0,1409940,0,0.0,6.5,G.Jesus,4
2,2,1997-12-19,9,459,False,True,0.0,0.0,10,0.38,226597,NaN,NaN,0,0,1,-1,208.8,202,45,NaN,NaN,2,2,0.0,0.0,0,1.21,0.05,3.91,0.15,2.70,23.09,0.88,0.10,Gabriel,0.0,789,266,22,0.84,3,False,108.2,135,36,3,False,584.6,108,40,2363,0,0,0,0,0,0,0,Hamstring injury - Expected back 01 Jun,2025-04-01T22:00:07.982867Z,61,70,6,p226597,0,0,NaN,0,NaN,226597.jpg,4.2,34,2,0,30.0,False,0,0.0,dos Santos Magalhães,12.2,34,13,False,NaN,28,1.07,i,1,3,2020-09-01,287.0,120,11,117,5093448,0,5431768,0,0.0,19.2,Gabriel,4
3,3,1999-06-11,14,343,False,True,100.0,100.0,7,0.34,219847,NaN,NaN,0,0,-3,3,269.0,161,18,NaN,NaN,1,4,1.5,1.7,1,1.65,0.08,11.18,0.54,9.53,19.32,0.93,0.46,Kai,0.2,413,40,20,0.96,9,False,144.8,81,17,4,False,467.6,155,18,1872,0,0,0,0,0,0,0,NaN,2025-02-12T09:00:08.755570Z,77,14,4,p219847,0,0,3.0,0,NaN,219847.jpg,4.2,36,10,0,80.0,False,0,0.0,Havertz,6.2,61,13,False,NaN,21,1.01,a,1,3,2023-06-28,711.0,34,15,97,2644355,0,3530099,0,0.0,12.6,Havertz,5
4,0,NaN,0,0,False,True,0.0,0.0,0,0.00,463748,NaN,NaN,0,0,0,0,0.0,599,57,NaN,NaN,0,1,0.0,0.0,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Karl,0.0,476,59,0,0.00,0,False,0.0,612,68,5,False,0.0,612,68,0,0,0,0,0,0,0,0,Loaned to Real Valladolid,2024-08-14T08:31:46.556082Z,40,667,53,p463748,0,0,NaN,0,NaN,463748.jpg,0.0,619,68,0,NaN,False,0,0.0,Hein,0.0,643,71,False,NaN,0,0.00,u,1,3,NaN,0.0,558,38,0,0,0,4643,0,0.0,0.0,Hein,0


In [618]:
print(elements_team.columns)
print(review_data.columns)

Index(['can_transact', 'can_select', 'chance_of_playing_next_round',
       'chance_of_playing_this_round', 'code_x', 'cost_change_event',
       'cost_change_event_fall', 'cost_change_start', 'cost_change_start_fall',
       'dreamteam_count',
       ...
       'team_division', 'unavailable', 'win', 'strength_overall_home',
       'strength_overall_away', 'strength_attack_home', 'strength_attack_away',
       'strength_defence_home', 'strength_defence_away', 'pulse_id'],
      dtype='object', length=118)
Index(['assists', 'birth_date', 'bonus', 'bps', 'can_select', 'can_transact',
       'chance_of_playing_next_round', 'chance_of_playing_this_round',
       'clean_sheets', 'clean_sheets_per_90',
       ...
       'threat_rank_type', 'total_points', 'transfers_in',
       'transfers_in_event', 'transfers_out', 'transfers_out_event',
       'value_form', 'value_season', 'web_name', 'yellow_cards'],
      dtype='object', length=103)


In [619]:

Name = {
    1:'Arsenal',
    2: 'Aston Villa',
    3: 'Bournemouth',
    4:'Brentford',
    5: 'Brighton',
    6: 'Chelsea',
    7: 'Crystal Palace',
    8: 'Everton',
    9: 'Fulham',
    10: 'Ipswich',
    11: 'Leicester',
    12: 'Liverpool',
    13: 'Man City',
    14: 'Man utd',
    15: 'Newcastle',
    16: "Nott'm Forest",
    17: 'Southampton',
    18:  'Spurs',
    19: 'West Ham',
    20: 'Wolves'
}
review_data['team_name'] = review_data['team'].map(Name)
print(review_data['team_name'])






0      Arsenal
1      Arsenal
2      Arsenal
3      Arsenal
4      Arsenal
        ...   
799     Wolves
800     Wolves
801     Wolves
802     Wolves
803     Wolves
Name: team_name, Length: 804, dtype: object


In [620]:

player_name_map = dict(zip(element_data['id'], element_data['web_name']))
review_data['web_name'] = review_data['id'].map(player_name_map)
review_data.head()





,assists,birth_date,bonus,bps,can_select,can_transact,chance_of_playing_next_round,chance_of_playing_this_round,clean_sheets,clean_sheets_per_90,code,corners_and_indirect_freekicks_order,corners_and_indirect_freekicks_text,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,creativity,creativity_rank,creativity_rank_type,direct_freekicks_order,direct_freekicks_text,dreamteam_count,element_type,ep_next,ep_this,event_points,expected_assists,expected_assists_per_90,expected_goal_involvements,expected_goal_involvements_per_90,expected_goals,expected_goals_conceded,expected_goals_conceded_per_90,expected_goals_per_90,first_name,form,form_rank,form_rank_type,goals_conceded,goals_conceded_per_90,goals_scored,has_temporary_code,ict_index,ict_index_rank,ict_index_rank_type,id,in_dreamteam,influence,influence_rank,influence_rank_type,minutes,mng_clean_sheets,mng_draw,mng_goals_scored,mng_loss,mng_underdog_draw,mng_underdog_win,mng_win,news,news_added,now_cost,now_cost_rank,now_cost_rank_type,opta_code,own_goals,penalties_missed,penalties_order,penalties_saved,penalties_text,photo,points_per_game,points_per_game_rank,points_per_game_rank_type,red_cards,region,removed,saves,saves_per_90,second_name,selected_by_percent,selected_rank,selected_rank_type,special,squad_number,starts,starts_per_90,status,team,team_code,team_join_date,threat,threat_rank,threat_rank_type,total_points,transfers_in,transfers_in_event,transfers_out,transfers_out_event,value_form,value_season,web_name,yellow_cards,team_name
0,0,NaN,0,0,False,True,0.0,0.0,0,0.00,438098,NaN,NaN,0,0,-1,1,0.0,795,343,NaN,NaN,0,3,0.0,0.0,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Fábio,0.0,773,331,0,0.00,0,False,0.0,795,343,1,False,0.0,795,343,0,0,0,0,0,0,0,0,Has joined Portuguese side FC Porto on loan for the 2024/25 season,2024-08-29T11:06:25.241953Z,54,134,82,p438098,0,0,NaN,0,NaN,438098.jpg,0.0,794,343,0,NaN,False,0,0.0,Ferreira Vieira,0.0,714,295,False,NaN,0,0.00,u,1,3,NaN,0.0,791,341,0,439,0,2825,0,0.0,0.0,Raya,0,Arsenal
1,2,1997-04-03,6,152,False,True,0.0,0.0,2,0.30,205651,NaN,NaN,0,0,-5,5,119.5,276,31,NaN,NaN,2,4,0.0,0.0,0,0.52,0.08,3.57,0.54,3.05,5.82,0.87,0.46,Gabriel,0.0,795,83,5,0.75,3,False,52.6,287,33,2,False,154.4,349,33,600,0,0,0,0,0,0,0,Knee injury - Expected back 01 Jun,2025-01-12T22:00:07.802845Z,65,43,20,p205651,0,0,NaN,0,NaN,205651.jpg,2.5,216,25,0,30.0,False,0,0.0,Fernando de Jesus,1.0,200,32,False,NaN,6,0.90,i,1,3,2022-07-04,255.0,134,32,42,1238437,0,1409940,0,0.0,6.5,Arrizabalaga,4,Arsenal
2,2,1997-12-19,9,459,False,True,0.0,0.0,10,0.38,226597,NaN,NaN,0,0,1,-1,208.8,202,45,NaN,NaN,2,2,0.0,0.0,0,1.21,0.05,3.91,0.15,2.70,23.09,0.88,0.10,Gabriel,0.0,789,266,22,0.84,3,False,108.2,135,36,3,False,584.6,108,40,2363,0,0,0,0,0,0,0,Hamstring injury - Expected back 01 Jun,2025-04-01T22:00:07.982867Z,61,70,6,p226597,0,0,NaN,0,NaN,226597.jpg,4.2,34,2,0,30.0,False,0,0.0,dos Santos Magalhães,12.2,34,13,False,NaN,28,1.07,i,1,3,2020-09-01,287.0,120,11,117,5093448,0,5431768,0,0.0,19.2,Hein,4,Arsenal
3,3,1999-06-11,14,343,False,True,100.0,100.0,7,0.34,219847,NaN,NaN,0,0,-3,3,269.0,161,18,NaN,NaN,1,4,1.5,1.7,1,1.65,0.08,11.18,0.54,9.53,19.32,0.93,0.46,Kai,0.2,413,40,20,0.96,9,False,144.8,81,17,4,False,467.6,155,18,1872,0,0,0,0,0,0,0,NaN,2025-02-12T09:00:08.755570Z,77,14,4,p219847,0,0,3.0,0,NaN,219847.jpg,4.2,36,10,0,80.0,False,0,0.0,Havertz,6.2,61,13,False,NaN,21,1.01,a,1,3,2023-06-28,711.0,34,15,97,2644355,0,3530099,0,0.0,12.6,Setford,5,Arsenal
4,0,NaN,0,0,False,True,0.0,0.0,0,0.00,463748,NaN,NaN,0,0,0,0,0.0,599,57,NaN,NaN,0,1,0.0,0.0,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Karl,0.0,476,59,0,0.00,0,False,0.0,612,68,5,False,0.0,612,68,0,0,0,0,0,0,0,0,Loaned to Real Valladolid,2024-08-14T08:31:46.556082Z,40,667,53,p463748,0,0,NaN,0,NaN,463748.jpg,0.0,619,68,0,NaN,False,0,0.0,Hein,0.0,643,71,False,NaN,0,0.00,u,1,3,NaN,0.0,558,38,0,0,0,4643,0,0.0,0.0,Gabriel,0,Arsenal


In [621]:

 merged_data = pd.merge(elements_team, review_data, left_on = ['name', 'web_name'], right_on = ['team_name', 'web_name'])
 with pd.option_context('Display.max_rows', None):
    print(merged_data[merged_data['web_name'] == 'Saliba'].iloc[0])

can_transact_x                                                           True
can_select_x                                                             True
chance_of_playing_next_round_x                                            NaN
chance_of_playing_this_round_x                                           None
code_x                                                                 462424
cost_change_event_x                                                         0
cost_change_event_fall_x                                                    0
cost_change_start_x                                                         0
cost_change_start_fall_x                                                    0
dreamteam_count_x                                                           0
element_type_x                                                              2
ep_next_x                                                                 4.5
ep_this_x                                                       

In [622]:
merged_data.set_index(['id_x'], inplace = True)

In [623]:
#next_gr = int(review_data.keys()[5].split('_')[0])

In [624]:
type_data = pd.DataFrame(fpl_data['element_types']).set_index(['id'])
type_data.head()

,plural_name,plural_name_short,singular_name,singular_name_short,squad_select,squad_min_select,squad_max_select,squad_min_play,squad_max_play,ui_shirt_specific,sub_positions_locked,element_count
id,,,,,,,,,,,,
1,Goalkeepers,GKP,Goalkeeper,GKP,2,None,None,1,1,True,[12],74
2,Defenders,DEF,Defender,DEF,5,None,None,3,5,False,[],219
3,Midfielders,MID,Midfielder,MID,5,None,None,2,5,False,[],297
4,Forwards,FWD,Forward,FWD,3,None,None,1,3,False,[],68


In [625]:
model = so.Model(name= 'single_peiod')

NOTE: Initialized model single_peiod.


In [626]:
players = merged_data.index.to_list()
element_types = type_data.index.to_list()
teams = team_data['name'].to_list()

In [627]:
squad = model.add_variables(players, name = 'squad', vartype = so.binary) #creates a boanry variable for squad 
lineup = model.add_variables(players, name = 'lineup', vartype = so.binary) #creates a boanry variable for squad 
captain = model.add_variables(players, name = 'captain', vartype = so.binary) #creates a boanry variable for squad 
vice_captain = model.add_variables(players, name = 'vice_captain', vartype = so.binary) #creates a boanry variable for squad 

In [628]:
squad_count = so.expr_sum(squad[p] for p in players) 

In [629]:
model.add_constraint(squad_count == 15, name = 'squad_count');

In [630]:
model.add_constraint(so.expr_sum(lineup[p] for p in players) == 11, name = 'lineup_cont') #must have 11 playeurs in lineup

sasoptpy.Constraint(lineup[1] + lineup[2] + lineup[3] + lineup[4] + lineup[5] + lineup[6] + lineup[7] + lineup[8] + lineup[9] + lineup[12] + lineup[13] + lineup[15] + lineup[16] + lineup[17] + lineup[18] + lineup[20] + lineup[21] + lineup[22] + lineup[23] + lineup[24] + lineup[25] + lineup[32] + lineup[33] + lineup[34] + lineup[35] + lineup[36] + lineup[37] + lineup[38] + lineup[39] + lineup[40] + lineup[41] + lineup[42] + lineup[43] + lineup[44] + lineup[45] + lineup[46] + lineup[47] + lineup[48] + lineup[49] + lineup[50] + lineup[51] + lineup[52] + lineup[53] + lineup[54] + lineup[55] + lineup[56] + lineup[57] + lineup[58] + lineup[66] + lineup[67] + lineup[68] + lineup[69] + lineup[70] + lineup[71] + lineup[72] + lineup[74] + lineup[75] + lineup[76] + lineup[77] + lineup[78] + lineup[79] + lineup[80] + lineup[81] + lineup[83] + lineup[84] + lineup[85] + lineup[101] + lineup[102] + lineup[103] + lineup[104] + lineup[105] + lineup[106] + lineup[107] + lineup[108] + lineup[109] + lineu

In [631]:
model.add_constraint(so.expr_sum(captain[p] for p in players) == 1, name ='captain_count')
model.add_constraint(so.expr_sum(vice_captain[p] for p in players) == 1, name ='vicecap_count')

sasoptpy.Constraint(vice_captain[1] + vice_captain[2] + vice_captain[3] + vice_captain[4] + vice_captain[5] + vice_captain[6] + vice_captain[7] + vice_captain[8] + vice_captain[9] + vice_captain[12] + vice_captain[13] + vice_captain[15] + vice_captain[16] + vice_captain[17] + vice_captain[18] + vice_captain[20] + vice_captain[21] + vice_captain[22] + vice_captain[23] + vice_captain[24] + vice_captain[25] + vice_captain[32] + vice_captain[33] + vice_captain[34] + vice_captain[35] + vice_captain[36] + vice_captain[37] + vice_captain[38] + vice_captain[39] + vice_captain[40] + vice_captain[41] + vice_captain[42] + vice_captain[43] + vice_captain[44] + vice_captain[45] + vice_captain[46] + vice_captain[47] + vice_captain[48] + vice_captain[49] + vice_captain[50] + vice_captain[51] + vice_captain[52] + vice_captain[53] + vice_captain[54] + vice_captain[55] + vice_captain[56] + vice_captain[57] + vice_captain[58] + vice_captain[66] + vice_captain[67] + vice_captain[68] + vice_captain[69] + v

In [632]:
model.add_constraints((lineup[p] <= squad[p] for p in players), name = 'lineup_squad_rel') #if a player is in the lineup he must be in the squad 
model.add_constraints((captain[p] <= lineup[p] for p in players), name = 'captain_lineup_rel')
model.add_constraints((vice_captain[p] <= lineup[p] for p in players), name='vicecap_lineup_rel')
model.add_constraints((captain[p] + vice_captain[p] <= 1 for p in players), name='cap_vc_rel') #cant pick both captain and vicecaptain together 

sasoptpy.ConstraintGroup([captain[1] + vice_captain[1] <=  1, captain[2] + vice_captain[2] <=  1, captain[3] + vice_captain[3] <=  1, captain[4] + vice_captain[4] <=  1, captain[5] + vice_captain[5] <=  1, captain[6] + vice_captain[6] <=  1, captain[7] + vice_captain[7] <=  1, captain[8] + vice_captain[8] <=  1, captain[9] + vice_captain[9] <=  1, captain[12] + vice_captain[12] <=  1, captain[13] + vice_captain[13] <=  1, captain[15] + vice_captain[15] <=  1, captain[16] + vice_captain[16] <=  1, captain[17] + vice_captain[17] <=  1, captain[18] + vice_captain[18] <=  1, captain[20] + vice_captain[20] <=  1, captain[21] + vice_captain[21] <=  1, captain[22] + vice_captain[22] <=  1, captain[23] + vice_captain[23] <=  1, captain[24] + vice_captain[24] <=  1, captain[25] + vice_captain[25] <=  1, captain[32] + vice_captain[32] <=  1, captain[33] + vice_captain[33] <=  1, captain[34] + vice_captain[34] <=  1, captain[35] + vice_captain[35] <=  1, captain[36] + vice_captain[36] <=  1, capt

In [633]:
lineup_type_count = {t:so.expr_sum(lineup[p] for p in players if merged_data.loc[p, 'element_type_x'] == t) for t in element_types } #creates a filter where t is the position and it makes a optimization sum to each lineup member only if their position matches the given one at the time (t filters from all posotion in the element_types column created earlier )

In [634]:
squad_type_count = {t:so.expr_sum(squad[p] for p in players if merged_data.loc[p, 'element_type_x'] == t) for t in element_types } 

In [635]:
model.add_constraints((lineup_type_count[t] == [type_data.loc[t, 'squad_min_play'], type_data.loc[t, 'squad_max_play']] for t in element_types), name = 'valid_formation') #cretaes a range of numbers which we can have in a lineup 

sasoptpy.ConstraintGroup([lineup[1] + lineup[2] + lineup[3] + lineup[4] + lineup[32] + lineup[33] + lineup[34] + lineup[35] + lineup[66] + lineup[67] + lineup[68] + lineup[69] + lineup[70] + lineup[101] + lineup[102] + lineup[103] + lineup[104] + lineup[105] + lineup[138] + lineup[139] + lineup[140] + lineup[141] + lineup[142] ==  1, lineup[5] + lineup[6] + lineup[7] + lineup[8] + lineup[9] + lineup[12] + lineup[13] + lineup[15] + lineup[36] + lineup[37] + lineup[38] + lineup[39] + lineup[40] + lineup[41] + lineup[42] + lineup[43] + lineup[44] + lineup[45] + lineup[46] + lineup[71] + lineup[72] + lineup[74] + lineup[75] + lineup[76] + lineup[77] + lineup[78] + lineup[79] + lineup[80] + lineup[106] + lineup[107] + lineup[108] + lineup[109] + lineup[110] + lineup[111] + lineup[112] + lineup[143] + lineup[144] + lineup[145] + lineup[146] + lineup[147] + lineup[148] + lineup[149] + lineup[637] ==  [3, 5], lineup[16] + lineup[17] + lineup[18] + lineup[20] + lineup[21] + lineup[22] + lineup[

In [636]:
model.add_constraints((squad_type_count[t] == type_data.loc[t, 'squad_select'] for t in element_types), name = 'valid_squad') #cretaes a range of numbers which we can have in a lineup 

sasoptpy.ConstraintGroup([squad[1] + squad[2] + squad[3] + squad[4] + squad[32] + squad[33] + squad[34] + squad[35] + squad[66] + squad[67] + squad[68] + squad[69] + squad[70] + squad[101] + squad[102] + squad[103] + squad[104] + squad[105] + squad[138] + squad[139] + squad[140] + squad[141] + squad[142] ==  2, squad[5] + squad[6] + squad[7] + squad[8] + squad[9] + squad[12] + squad[13] + squad[15] + squad[36] + squad[37] + squad[38] + squad[39] + squad[40] + squad[41] + squad[42] + squad[43] + squad[44] + squad[45] + squad[46] + squad[71] + squad[72] + squad[74] + squad[75] + squad[76] + squad[77] + squad[78] + squad[79] + squad[80] + squad[106] + squad[107] + squad[108] + squad[109] + squad[110] + squad[111] + squad[112] + squad[143] + squad[144] + squad[145] + squad[146] + squad[147] + squad[148] + squad[149] + squad[637] ==  5, squad[16] + squad[17] + squad[18] + squad[20] + squad[21] + squad[22] + squad[23] + squad[24] + squad[25] + squad[47] + squad[48] + squad[49] + squad[50] + 

In [637]:
budget = 120  # in millions

price = so.expr_sum((merged_data.loc[p, 'now_cost_x'] / 10) * squad[p] for p in players )
model.add_constraint(price <= budget, name='budget_limit')


sasoptpy.Constraint(5.5 * squad[1] + 4.5 * squad[2] + 4.0 * squad[3] + 4.0 * squad[4] + 6.0 * squad[5] + 6.0 * squad[6] + 5.5 * squad[7] + 5.5 * squad[8] + 5.5 * squad[9] + 5.0 * squad[12] + 4.0 * squad[13] + 4.0 * squad[15] + 10.0 * squad[16] + 8.0 * squad[17] + 7.0 * squad[18] + 7.0 * squad[20] + 6.5 * squad[21] + 6.0 * squad[22] + 5.5 * squad[23] + 5.5 * squad[24] + 5.5 * squad[25] + 5.0 * squad[32] + 4.5 * squad[33] + 4.0 * squad[34] + 4.0 * squad[35] + 4.5 * squad[36] + 4.5 * squad[37] + 4.5 * squad[38] + 4.5 * squad[39] + 4.5 * squad[40] + 4.5 * squad[41] + 4.0 * squad[42] + 4.0 * squad[43] + 4.0 * squad[44] + 4.0 * squad[45] + 4.0 * squad[46] + 7.0 * squad[47] + 6.0 * squad[48] + 5.5 * squad[49] + 5.5 * squad[50] + 5.5 * squad[51] + 5.5 * squad[52] + 5.5 * squad[53] + 5.5 * squad[54] + 5.0 * squad[55] + 5.0 * squad[56] + 5.0 * squad[57] + 5.0 * squad[58] + 4.5 * squad[66] + 4.5 * squad[67] + 4.0 * squad[68] + 4.0 * squad[69] + 4.0 * squad[70] + 5.0 * squad[71] + 4.5 * squad[72] 

In [638]:
model.add_constraints((so.expr_sum(squad[p] for p in players if merged_data.loc[p, 'team_name'] == t) <=3 for t in teams), name= 'team_limit')

sasoptpy.ConstraintGroup([squad[1] + squad[2] + squad[3] + squad[4] + squad[5] + squad[6] + squad[7] + squad[8] + squad[9] + squad[12] + squad[13] + squad[15] + squad[16] + squad[17] + squad[18] + squad[20] + squad[21] + squad[22] + squad[23] + squad[24] + squad[25] <=  3, squad[32] + squad[33] + squad[34] + squad[35] + squad[36] + squad[37] + squad[38] + squad[39] + squad[40] + squad[41] + squad[42] + squad[43] + squad[44] + squad[45] + squad[46] + squad[47] + squad[48] + squad[49] + squad[50] + squad[51] + squad[52] + squad[53] + squad[54] + squad[55] + squad[56] + squad[57] + squad[58] <=  3, - 3 <=  3, squad[66] + squad[67] + squad[68] + squad[69] + squad[70] + squad[71] + squad[72] + squad[74] + squad[75] + squad[76] + squad[77] + squad[78] + squad[79] + squad[80] + squad[81] + squad[83] + squad[84] + squad[85] <=  3, squad[101] + squad[102] + squad[103] + squad[104] + squad[105] + squad[106] + squad[107] + squad[108] + squad[109] + squad[110] + squad[111] + squad[112] <=  3, squa

In [639]:
total_points = so.expr_sum( float(merged_data.loc[p, 'ep_next_x']) * (lineup[p] + captain[p] + 0.1 * vice_captain[p]) for p in players if pd.notna(merged_data.loc[p, 'ep_next_x']))

total_points

#Im not confident about this bit here. i feel like ive messed up 


sasoptpy.Expression(exp = 4.5 * lineup[1] + 4.5 * captain[1] + 0.45 * vice_captain[1] + 2.8 * lineup[2] + 2.8 * captain[2] + 0.28 * vice_captain[2] + 1.5 * lineup[3] + 1.5 * captain[3] + 0.15 * vice_captain[3] + 1.5 * lineup[4] + 1.5 * captain[4] + 0.15 * vice_captain[4] + 4.5 * lineup[5] + 4.5 * captain[5] + 0.45 * vice_captain[5] + 4.5 * lineup[6] + 4.5 * captain[6] + 0.45 * vice_captain[6] + 3.8 * lineup[7] + 3.8 * captain[7] + 0.38 * vice_captain[7] + 3.8 * lineup[8] + 3.8 * captain[8] + 0.38 * vice_captain[8] + 3.8 * lineup[9] + 3.8 * captain[9] + 0.38 * vice_captain[9] + 3.1 * lineup[12] + 3.1 * captain[12] + 0.31 * vice_captain[12] + 1.5 * lineup[13] + 1.5 * captain[13] + 0.15 * vice_captain[13] + 1.5 * lineup[15] + 1.5 * captain[15] + 0.15 * vice_captain[15] + 3.5 * lineup[16] + 3.5 * captain[16] + 0.35 * vice_captain[16] + 3.1 * lineup[17] + 3.1 * captain[17] + 0.31 * vice_captain[17] + 2.8 * lineup[18] + 2.8 * captain[18] + 0.28 * vice_captain[18] + 2.8 * lineup[20] + 2.8 * c

In [640]:
model.set_objective(-total_points, sense='N',name='total_xp')

sasoptpy.Expression(exp = - 4.5 * lineup[1] - 4.5 * captain[1] - 0.45 * vice_captain[1] - 2.8 * lineup[2] - 2.8 * captain[2] - 0.28 * vice_captain[2] - 1.5 * lineup[3] - 1.5 * captain[3] - 0.15 * vice_captain[3] - 1.5 * lineup[4] - 1.5 * captain[4] - 0.15 * vice_captain[4] - 4.5 * lineup[5] - 4.5 * captain[5] - 0.45 * vice_captain[5] - 4.5 * lineup[6] - 4.5 * captain[6] - 0.45 * vice_captain[6] - 3.8 * lineup[7] - 3.8 * captain[7] - 0.38 * vice_captain[7] - 3.8 * lineup[8] - 3.8 * captain[8] - 0.38 * vice_captain[8] - 3.8 * lineup[9] - 3.8 * captain[9] - 0.38 * vice_captain[9] - 3.1 * lineup[12] - 3.1 * captain[12] - 0.31 * vice_captain[12] - 1.5 * lineup[13] - 1.5 * captain[13] - 0.15 * vice_captain[13] - 1.5 * lineup[15] - 1.5 * captain[15] - 0.15 * vice_captain[15] - 3.5 * lineup[16] - 3.5 * captain[16] - 0.35 * vice_captain[16] - 3.1 * lineup[17] - 3.1 * captain[17] - 0.31 * vice_captain[17] - 2.8 * lineup[18] - 2.8 * captain[18] - 0.28 * vice_captain[18] - 2.8 * lineup[20] - 2.8 *

In [641]:
model.export_mps('single_period.mps')

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sasoptpy/interface/format/mps_format.py:188: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[f] = df[f].replace('', np.nan)


In [642]:
command = 'cbc single_period.mps -solve -solu solution_sp.txt'
!{command}

Welcome to the CBC MILP Solver
Version: Devel (unstable)
Build Date: Jul  4 2025
command line - single_period.mps -solve -solu solution_sp.txt (default strategy 1)
At line 1 NAME    single_peiod
At line 2 ROWS
At line 409 COLUMNS
At line 1340 RHS
At line 1404 RANGES
At line 1408 BOUNDS
At line 1781 ENDATA
Problem single_peiod has 405 rows, 372 columns and 1488 elements
Coin0008I single_peiod read with 0 errors
Problem is infeasible - 0.000379 seconds
Total time (CPU seconds):       0.00113   (Wallclock seconds):       0.00159192


In [643]:
for v in model.get_variables():
    v.set_value(0)

In [644]:
with open('solution_sp.txt', 'r') as f:
    for line in f:
        if 'objective value' in line or line.strip() == '':
            continue

        words = line.split()
        full_name = words[1]
        value = float(words[-1])

        # Parse variable name and index using regex, e.g., squad[7] -> name=squad, index=7
        match = re.match(r'(\w+)\[(\d+)\]', full_name)
        if match:
            var_base = match.group(1)
            var_index = int(match.group(2))

            # Access variable dictionary
            if var_base == 'squad':
                squad[var_index].set_value(value)
            elif var_base == 'lineup':
                lineup[var_index].set_value(value)
            elif var_base == 'captain':
                captain[var_index].set_value(value)
            elif var_base == 'vice_captain':
                vice_captain[var_index].set_value(value)
            else:
                print(f"[Warning] Unknown variable base: {var_base}")
        else:
            print(f"[Warning] Could not parse variable name: {full_name}")

[Warning] Could not parse variable name: 34
[Warning] Could not parse variable name: 279


In [645]:
picks = []

for p in players:
    if squad[p].get_value() > 0.5:
        lp = elements_team.loc[p]

        is_captain = 1 if captain[p].get_value() > 0.5 else 0
        is_lineup = 1 if lineup[p].get_value() > 0.5 else 0
        is_vice = 1 if vice_captain[p].get_value() > 0.5 else 0

        position = type_data.loc[lp['element_type'], 'singular_name_short']

        # Ensure ep_next is a float before rounding
        xp = round(float(lp['ep_next']), 2) if pd.notna(lp['ep_next']) else 0.0

        picks.append([
            lp['web_name'],
            position,
            lp['element_type'],
            lp['name'],
            lp['now_cost'] / 10,
            xp,
            is_lineup,
            is_captain,
            is_vice
        ])

# Fixed typo: 'column' ➝ 'columns'
picks_df = pd.DataFrame(picks, columns=[ 'name', 'pos', 'type', 'team', 'price', 'xp', 'lineup', 'captain', 'vice_captain' ]).sort_values(by=['lineup','type','xp'],ascending=[False,True,True])


In [646]:
picks_df

,name,pos,type,team,price,xp,lineup,captain,vice_captain


In [647]:
total_xp = so.expr_sum( float(merged_data.loc[p, 'ep_next_x'] or 0) * (lineup[p] + captain[p]) for p in players ).get_value()
total_xp

-38.681778404339994